# Importação das bibliotecas e carregamento dos dados

In [132]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns, petl as etl
from petl import convert, rename, select

df = pd.read_csv('creditos-ativos.csv', delimiter=';', encoding='latin1')

df.head()

,ano_mes,situacao,tipo_contribuinte,cnae_secao_cod,cnae_secao_descr,nat_juridica,uf,grupo_tributo,vl_total
0,2023-07,DEVEDOR,PF,NaN,NaN,NaN,AC,CONTRIB. PREV.,"3861807,15"
1,2023-07,DEVEDOR,PF,NaN,NaN,NaN,AC,IRPF,"52726729,42"
2,2023-07,DEVEDOR,PF,NaN,NaN,NaN,AC,IRRF,"59873,2"
3,2023-07,DEVEDOR,PF,NaN,NaN,NaN,AC,ITR,"8171734,62"
4,2023-07,DEVEDOR,PF,NaN,NaN,NaN,AC,MULTAS,"994572,04"


# Análise exploratória

In [133]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 694467 entries, 0 to 694466
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   ano_mes            694467 non-null  object
 1   situacao           694467 non-null  object
 2   tipo_contribuinte  694467 non-null  object
 3   cnae_secao_cod     677627 non-null  object
 4   cnae_secao_descr   677627 non-null  object
 5   nat_juridica       684678 non-null  object
 6   uf                 694376 non-null  object
 7   grupo_tributo      694467 non-null  object
 8   vl_total           694467 non-null  object
dtypes: object(9)
memory usage: 47.7+ MB


In [134]:
df.describe()

,ano_mes,situacao,tipo_contribuinte,cnae_secao_cod,cnae_secao_descr,nat_juridica,uf,grupo_tributo,vl_total
count,694467,694467,694467,677627,677627,684678,694376,694467,694467
unique,11,4,2,19,19,75,28,20,644373
top,2023-07,DEVEDOR,PJ,M,"ATIVIDADES PROFISSIONAIS, CIENTÃFICAS E TÃCN...",SOCIEDADE EMPRESÃRIA LIMITADA,SP,MULTAS,"250,0"
freq,63871,285122,684690,59646,59646,171594,59760,104451,1556


In [135]:
df.isnull().sum()

ano_mes                  0
situacao                 0
tipo_contribuinte        0
cnae_secao_cod       16840
cnae_secao_descr     16840
nat_juridica          9789
uf                      91
grupo_tributo            0
vl_total                 0
dtype: int64

# Início do processo de ETL

## Extração

In [136]:
df.head()

,ano_mes,situacao,tipo_contribuinte,cnae_secao_cod,cnae_secao_descr,nat_juridica,uf,grupo_tributo,vl_total
0,2023-07,DEVEDOR,PF,NaN,NaN,NaN,AC,CONTRIB. PREV.,"3861807,15"
1,2023-07,DEVEDOR,PF,NaN,NaN,NaN,AC,IRPF,"52726729,42"
2,2023-07,DEVEDOR,PF,NaN,NaN,NaN,AC,IRRF,"59873,2"
3,2023-07,DEVEDOR,PF,NaN,NaN,NaN,AC,ITR,"8171734,62"
4,2023-07,DEVEDOR,PF,NaN,NaN,NaN,AC,MULTAS,"994572,04"


## Transformação

In [137]:
df.dtypes

ano_mes              object
situacao             object
tipo_contribuinte    object
cnae_secao_cod       object
cnae_secao_descr     object
nat_juridica         object
uf                   object
grupo_tributo        object
vl_total             object
dtype: object

In [138]:
# Convertendo valor total para float

df['vl_total'] = df['vl_total'].str.replace(',', '.').astype(float)

In [139]:
df['vl_total'].head()

0     3861807.15
1    52726729.42
2       59873.20
3     8171734.62
4      994572.04
Name: vl_total, dtype: float64

In [158]:
# Mapeando situação para float

lista_situacoes = df['situacao'].unique().tolist()

df['situacao'] = df['situacao'].map(dict(zip(lista_situacoes, range(len(lista_situacoes))))).astype(float)

In [140]:
# Mapenado grupo tributo para float

lista_tributos = df['grupo_tributo'].unique().tolist()

df['grupo_tributo'] = df['grupo_tributo'].map(dict(zip(lista_tributos, range(len(lista_tributos))))).astype(float)

In [159]:
# Mapeando uf para float

lista_ufs = df['uf'].unique().tolist()

df['uf'] = df['uf'].map(dict(zip(lista_ufs, range(len(lista_ufs))))).astype(float)

In [142]:
# Convertendo tipo contribuent para 0 e 1

df['tipo_contribuinte'] = df['tipo_contribuinte'].map({'PF': 0, 'PJ': 1}).astype(int)

In [143]:
df['tipo_contribuinte'].value_counts()

tipo_contribuinte
1    684690
0      9777
Name: count, dtype: int64

In [144]:
# Removendo colunas desnecessárias (CNAE e natureza jurídica)

df.drop(['cnae_secao_cod', 'cnae_secao_descr', 'nat_juridica'], axis=1, inplace=True)

In [145]:
# Removendo colunas com valores nulos

df.dropna(inplace=True)

In [146]:
df

,ano_mes,situacao,tipo_contribuinte,uf,grupo_tributo,vl_total
0,2023-07,DEVEDOR,0,1.0,0.0,3861807.15
1,2023-07,DEVEDOR,0,1.0,1.0,52726729.42
2,2023-07,DEVEDOR,0,1.0,2.0,59873.20
3,2023-07,DEVEDOR,0,1.0,3.0,8171734.62
4,2023-07,DEVEDOR,0,1.0,4.0,994572.04
...,...,...,...,...,...,...
694462,2022-09,PARCELADO,1,26.0,0.0,2494447.86
694463,2022-09,PARCELADO,1,26.0,10.0,867944.64
694464,2022-09,PARCELADO,1,26.0,8.0,1574585.68
694465,2022-09,PARCELADO,1,26.0,2.0,289185.77


## Load

In [147]:
df.to_csv('creditos-ativos-etl.csv', index=False)